In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import dotenv
import os
import gc
import cv2

dotenv.load_dotenv('.env')

True

In [2]:
train = pd.read_csv('raw/train.csv').set_index('id')
test = pd.read_csv('raw/sample_submission.csv').set_index('id')
depths = pd.read_csv('raw/depths.csv').set_index('id')

train = pd.merge(train, depths, left_index=True, right_index=True, how='inner')
train.shape, test.shape, depths.shape

((4000, 2), (18000, 1), (22000, 1))

In [0]:
IMG_SIZE = 128

In [4]:
X_train = [cv2.resize(cv2.imread("./raw/images/{}.png".format(idx), cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE)) for idx in tqdm(train.index)]
X_train = np.expand_dims(np.array(X_train), -1)
X_train = X_train / 255
X_train = X_train.astype(np.float32)

y_train = [cv2.resize(cv2.imread("./raw/masks/{}.png".format(idx), cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE)) for idx in tqdm(train.index)]
y_train = np.expand_dims(np.array(y_train), -1)
y_train = y_train / 255
y_train = y_train.astype(np.float32)


X_test = [cv2.resize(cv2.imread("./raw/images/{}.png".format(idx), cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE)) for idx in tqdm(test.index)]
X_test = np.expand_dims(np.array(X_test), -1)
X_test = X_test / 255
X_test = X_test.astype(np.float32)


X_train.shape, y_train.shape, X_test.shape

100%|██████████| 18000/18000 [00:06<00:00, 2727.82it/s]


((4000, 128, 128, 1), (4000, 128, 128, 1), (18000, 128, 128, 1))

In [5]:
from tensorflow import keras
from tensorflow.keras.layers import *
import tensorflow as tf
# from keras import Model
# from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
# from keras.models import load_model
# from keras.optimizers import Adam
# from keras.utils.vis_utils import plot_model
# from keras.preprocessing.image import ImageDataGenerator
# from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, Concatenate, Dropout, BatchNormalization, SpatialDropout2D, UpSampling2D, Activation, Add

# from keras import backend as K



def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score
  
  

# define iou or jaccard loss function
def jaccard_loss(y_true, y_pred):
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true * y_pred)
    
    epsilon = tf.keras.backend.epsilon()
    
    score = (intersection + epsilon) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection + epsilon)
    return 1 - score


# input: (x, 101, 101, 1), (x, 101, 101, 1)
# output: (x,)
def batch_iou(y_true, y_pred):
    pred_ = tf.round(y_pred)
    truth_ = y_true
    
    i = tf.reduce_sum(pred_ * truth_, axis=[1,2,3])
    u = tf.reduce_sum((pred_ + truth_) - (pred_ * truth_), axis=[1,2,3])
    
    epsilon = tf.keras.backend.epsilon()
    return (i + epsilon) / (u + epsilon)

# input: (x,)
# output: ()
def mean_iou(y_true, y_pred):
    ious = batch_iou(y_true, y_pred)
    ious = tf.reshape(ious, (-1,1))
    ious = tf.tile(ious, (1,10))
    thresholds = tf.range(0.5, 1.0, delta=0.05)
    
    return tf.reduce_mean(tf.to_float(ious >= thresholds))


def lovasz_grad(gt_sorted):
    """
    Computes gradient of the Lovasz extension w.r.t sorted errors
    See Alg. 1 in paper
    """
    gts = tf.reduce_sum(gt_sorted)
    intersection = gts - tf.cumsum(gt_sorted)
    union = gts + tf.cumsum(1. - gt_sorted)
    jaccard = 1. - intersection / union
    jaccard = tf.concat((jaccard[0:1], jaccard[1:] - jaccard[:-1]), 0)
    return jaccard
  
  
  
def lovasz_hinge(logits, labels, per_image=True, ignore=None):
    """
    Binary Lovasz hinge loss
      logits: [B, H, W] Variable, logits at each pixel (between -\infty and +\infty)
      labels: [B, H, W] Tensor, binary ground truth masks (0 or 1)
      per_image: compute the loss per image instead of per batch
      ignore: void class id
    """
    if per_image:
        def treat_image(log_lab):
            log, lab = log_lab
            log, lab = tf.expand_dims(log, 0), tf.expand_dims(lab, 0)
            log, lab = flatten_binary_scores(log, lab, ignore)
            return lovasz_hinge_flat(log, lab)
        losses = tf.map_fn(treat_image, (logits, labels), dtype=tf.float32)
        loss = tf.reduce_mean(losses)
    else:
        loss = lovasz_hinge_flat(*flatten_binary_scores(logits, labels, ignore))
    return loss
  
  
def lovasz_hinge_flat(logits, labels):
    """
    Binary Lovasz hinge loss
      logits: [P] Variable, logits at each prediction (between -\infty and +\infty)
      labels: [P] Tensor, binary ground truth labels (0 or 1)
      ignore: label to ignore
    """

    def compute_loss():
        labelsf = tf.cast(labels, logits.dtype)
        signs = 2. * labelsf - 1.
        errors = 1. - logits * tf.stop_gradient(signs)
        errors_sorted, perm = tf.nn.top_k(errors, k=tf.shape(errors)[0], name="descending_sort")
        gt_sorted = tf.gather(labelsf, perm)
        grad = lovasz_grad(gt_sorted)
        #loss = tf.tensordot(tf.nn.relu(errors_sorted), tf.stop_gradient(grad), 1, name="loss_non_void")
        loss = tf.tensordot(tf.nn.elu(errors_sorted), tf.stop_gradient(grad), 1, name="loss_non_void")
        return loss

    # deal with the void prediction case (only void pixels)
    loss = tf.cond(tf.equal(tf.shape(logits)[0], 0),
                   lambda: tf.reduce_sum(logits) * 0.,
                   compute_loss,
                   strict=True,
                   name="loss"
                   )
    return loss
  
  
def flatten_binary_scores(scores, labels, ignore=None):
    """
    Flattens predictions in the batch (binary case)
    Remove labels equal to 'ignore'
    """
    scores = tf.reshape(scores, (-1,))
    labels = tf.reshape(labels, (-1,))
    if ignore is None:
        return scores, labels
    valid = tf.not_equal(labels, ignore)
    vscores = tf.boolean_mask(scores, valid, name='valid_scores')
    vlabels = tf.boolean_mask(labels, valid, name='valid_labels')
    return vscores, vlabels

def lovasz_loss(y_true, y_pred):
    y_true, y_pred = K.cast(K.squeeze(y_true, -1), 'int32'), K.cast(K.squeeze(y_pred, -1), 'float32')
    #logits = K.log(y_pred / (1. - y_pred))
    logits = y_pred #Jiaxin
    loss = lovasz_hinge(logits, y_true, per_image = True, ignore = None)
    return loss
  

# def iou_bce_loss(y_true, y_pred):
#     return 0.5 * keras.losses.binary_crossentropy(y_true, y_pred) + 0.5 * jaccard_loss(y_true, y_pred)
  
  
# def mean_iou(y_true, y_pred):
#     return tf.reduce_mean(tf.to_float(tf.greater(tf.round(y_true), y_pred)))
    

# def mean_iou(y_true, y_pred):
#     i = tf.round(y_pred) * tf.round(y_true)
#     o = tf.round(y_pred) * tf.round(y_true)
#     mean_iou = tf.metrics.mean_iou(y_true, tf.round(y_pred), 1)
#     print(mean_iou)
#     print(tf.tile(mean_iou, 10))
#     print(mean_iou[0])
#     return mean_iou[0]
#     return tf.reduce_mean(tf.tile(mean_iou, 10))
  
# def mean_iou(y_true, y_pred):
#     prec = []
#     for t in np.arange(0.5, 1.0, 0.05):
#         y_pred_ = tf.to_int32(y_pred > t)
#         score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
#         K.get_session().run(tf.local_variables_initializer())
#         with tf.control_dependencies([up_opt]):
#             score = tf.identity(score)
#         prec.append(score)
#     return K.mean(K.stack(prec), axis=0)


# def convolution_block(x, filters, size):
#     return x



  
# def residual_block(blockInput, filters):
#     x = Conv2D(filters, 3, strides=(1,1), padding='same', activation=None)(blockInput)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     x = Conv2D(filters, 3, strides=(1,1), padding='same', activation=None)(x)
#     x = Add()([x, blockInput])
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     return x
  
  
# def res_block(i, filters=16):
#     n1 = Conv2D(filters, 1, activation='relu', padding='same')(i)
#     n3 = Conv2D(filters, 3, activation='relu', padding='same')(i)
#     n5 = Conv2D(filters, 5, activation='relu', padding='same')(i)
#     n = Concatenate()([n1, n3, n5])
#     n = BatchNormalization()(n)
    
#     n1 = Conv2D(filters, 1, activation='relu', padding='same')(n)
#     n3 = Conv2D(filters, 3, activation='relu', padding='same')(n)
#     n5 = Conv2D(filters, 5, activation='relu', padding='same')(n)
#     n = Concatenate()([n1, n3, n5])
#     n = BatchNormalization()(n)
    
# #     n = Conv2D(1, 1, activation='relu', padding='same')(n)
# #     n = Concatenate()([i, n])
# #     n = BatchNormalization()(n)
#     return n

  
# def level_block(inp, dim, depth, inc, acti, do, bn, mp, up, res):
#     if depth > 0:
#         n = conv_block(m, dim, acti, bn, res)
#         m = MaxPooling2D()(n) if mp else Conv2D(dim, 3, strides=2, padding='same')(n)
#         m = level_block(m, int(inc*dim), depth-1, inc, acti, do, bn, mp, up, res)
#         if up:
#             m = UpSampling2D()(m)
#             m = Conv2D(dim, 2, activation=acti, padding='same')(m)
#         else:
# #             padding = ['valid', 'same','same','valid','same','valid']
#             padding = ['same','same','valid','same','valid']
#             m = Conv2DTranspose(dim, 3, strides=2, activation=acti, padding=padding[depth-1])(m)
#         n = Concatenate()([n, m])
#         m = conv_block(n, dim, acti, bn, res)
#     else:
#         m = conv_block(m, dim, acti, bn, res, do=do)
#     return m

  
# def ResNet():
#     filter_size = 8
#     i = Input(shape=(img_size_target,img_size_target,1))
#     o = Conv2D(filter_size, 1, strides=(1,1), padding='same', activation='relu')(i)
#     for depth in tqdm(range(128)):
#       o = residual_block(o, filter_size)
#     o = Conv2D(1, 1, strides=(1,1), padding='same', activation='sigmoid')(o)
#     return Model(inputs=i, outputs=o)
  
  
# def ResNet_2():
#     resnet = keras.applications.resnet50.ResNet50(include_top=False, pooling=None, input_shape=(IMG_SIZE, IMG_SIZE, 3))
#     o = Conv2D(1, 1, strides=(1,1), activation='sigmoid')(resnet.output)
#     return Model(inputs=resnet.input, outputs=o)
  
def res_block(inputs, filters):
    inputs = BatchNormalization()(inputs)
    conv = Conv2D(filters, 3, strides=1, padding='same', activation='relu')(inputs)
    
    conv = BatchNormalization()(conv)
    conv = Conv2D(filters, 3, strides=1, padding='same', activation='relu')(conv)
    
    conv = Add()([conv, inputs])
    
    conv = BatchNormalization()(conv)
    return Conv2D(filters, 3, strides=2, padding='same', activation='relu')(conv)
  

def res_block_r(inputs, filters, res):
    inputs = Add()([inputs, res])
    inputs = BatchNormalization()(inputs)
    conv = Conv2DTranspose(filters, 3, strides=1, padding='same', activation='relu')(inputs)
    
    conv = BatchNormalization()(conv)
    conv = Conv2DTranspose(filters, 3, strides=1, padding='same', activation='relu')(conv)
    
    conv = Add()([conv, inputs])
    
    conv = BatchNormalization()(conv)
    return Conv2DTranspose(filters, 3, strides=2, padding='same', activation='relu')(conv)
  
  
# def build_model(start_channels=8, depth=6):
#     inp = Input(shape=(IMG_SIZE, IMG_SIZE, 1))
#     l = BatchNormalization()(inp)
#     l = Conv2D(start_channels, 3, strides=1, padding='same', activation='relu')(l)
    
#     layers = [l]
    
#     for i in range(depth):
#       layers.append(res_block(layers[-1], start_channels * (2 ** i)))
      
      
#     for i in range(depth):
#       layers.append(res_block_r(layers[-1], start_channels * (2 ** (depth - 1 - i)), layers[-2*i-1]))
      
#     l = BatchNormalization()(layers[-1])
#     out = Conv2D(1, 1, strides=(1,1), padding='same', activation='sigmoid')(l)
    
#     return Model(inputs=inp, outputs=out)

def UNet(inputs, channels, depth):
    layers = [inputs]
    
    for i in range(depth):
      layers.append(res_block(layers[-1], channels))
      
      
    for i in range(depth):
      layers.append(res_block_r(layers[-1], channels, layers[-2*i-1]))
    
    return layers[-1]
    
    

def build_model(channels=64, depth=6):
    inp = keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 1))
    l = BatchNormalization()(inp)
    l = Conv2D(channels, 3, strides=1, padding='same', activation='relu')(l)
    
    l = UNet(l, channels, depth)
#     l = UNet(l, channels, depth)
    
    l = BatchNormalization()(l)
    out = Conv2D(1, 1, strides=(1,1), padding='same', activation='sigmoid')(l)
    
    return keras.Model(inputs=inp, outputs=out)


  
build_model().summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 128, 1)  4           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 64) 640         batch_normalization[0][0]        
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 128, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (

In [0]:
from albumentations import (
    PadIfNeeded,
    HorizontalFlip,
    VerticalFlip,    
    CenterCrop,    
    Crop,
    Compose,
    Transpose,
    RandomRotate90,
    ElasticTransform,
    GridDistortion, 
    OpticalDistortion,
    RandomSizedCrop,
    OneOf,
    CLAHE,
    RandomContrast,
    RandomGamma,
    RandomBrightness
)

def random_h_flip(image, mask):
  aug = HorizontalFlip(p=0.5)
  augmented = aug(image=image, mask=mask)
  return augmented['image'], augmented['mask']


def random_v_flip(image, mask):
  aug = VerticalFlip(p=0.5)
  augmented = aug(image=image, mask=mask)
  return augmented['image'], augmented['mask']


def random_rotate(image, mask):
  aug = RandomRotate90(p=1)
  augmented = aug(image=image, mask=mask)
  return augmented['image'], augmented['mask']


def random_aug(image, mask):
  image, mask = random_h_flip(image, mask)
  image, mask = random_v_flip(image, mask)
  image, mask = random_rotate(image, mask)
  return image, mask


SEED = 42

def _random_flip_left_right(image, mask):
  image = tf.image.random_flip_left_right(image, seed=SEED)
  mask = tf.image.random_flip_left_right(mask, seed=SEED)
  return image, mask

def _random_flip_up_down(image, mask):
  image = tf.image.random_flip_up_down(image, seed=SEED)
  mask = tf.image.random_flip_up_down(mask, seed=SEED)
  return image, mask

In [7]:
from tensorflow.keras.callbacks import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def build_train_generator(X_train, y_train, batch_size=32):
    # Ref: https://keras.io/preprocessing/image/
    # we create two instances with the same arguments
    data_gen_args = dict(
#       rotation_range=360.,
#       width_shift_range=0.1,
#       height_shift_range=0.1,
      fill_mode='reflect',
      horizontal_flip=True,
      vertical_flip=True
    )
    image_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)

    # Provide the same seed and keyword arguments to the fit and flow methods
    seed = 1
    image_datagen.fit(X_train, augment=True, seed=seed)
    mask_datagen.fit(y_train, augment=True, seed=seed)
    
    image_generator = image_datagen.flow(X_train, batch_size=batch_size, seed=seed)
    mask_generator = mask_datagen.flow(y_train, batch_size=batch_size, seed=seed)

    # combine generators into one which yields image and masks
    return zip(image_generator, mask_generator)

  

def cv(X_train, y_train, X_test, fold, fast=False):
    from sklearn.model_selection import KFold
    kf = KFold(n_splits=fold, shuffle=True, random_state=42)
    
    loss = []
    metric = []
    pred_test = np.zeros(X_test.shape)
    fold_index = 0
    
    start_ch = 32
    depth = 5
         
    
    for train_index, test_index in kf.split(X_train):      
#       model = UNet((img_size_target,img_size_target,1),start_ch=start_ch,depth=depth,batchnorm=True, dropout=False)
      model = build_model(channels=8)
  
      resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu='kaggle-tpu')
      strategy = tf.contrib.tpu.TPUDistributionStrategy(resolver)
      model = tf.contrib.tpu.keras_to_tpu_model(model, strategy=strategy)
      session_master = resolver.master()
  
      model.compile(loss='binary_crossentropy', optimizer="adam", metrics=[mean_iou, "binary_crossentropy"])
    
      EPOCHS = 20000
      BATCH_SIZE = 64

      def create_data_fn(index, training=False):
        def data_fn():
          dataset = tf.data.Dataset.from_tensor_slices((X_train[index], y_train[index]))
          if training:
            dataset = dataset.map(_random_flip_left_right).map(_random_flip_up_down)
          dataset = dataset.shuffle(len(X_train[index])).repeat(EPOCHS).batch(BATCH_SIZE, drop_remainder=True)
          return dataset
        return data_fn
          

      early_stopping = EarlyStopping(patience=10, verbose=1)
      model_checkpoint = ModelCheckpoint("./model/keras_%d.model"%fold_index, save_best_only=True, verbose=1)
      reduce_lr = ReduceLROnPlateau(factor=0.5, patience=5, min_lr=0.00001, verbose=1)

      history = model.fit(
          create_data_fn(train_index, training=True),
          steps_per_epoch=int(np.ceil(len(X_train[train_index]) / BATCH_SIZE)),
          validation_data=create_data_fn(test_index, training=False),
          validation_steps=int(np.ceil(len(X_train[test_index]) / BATCH_SIZE)),
          epochs=EPOCHS
#           callbacks=[early_stopping, model_checkpoint, reduce_lr],
#           callbacks=[early_stopping, reduce_lr]
      )
      
      best_idx = np.argmin(history.history['val_loss'])
      val_loss = history.history['val_loss'][best_idx]
      val_mean_iou = history.history['val_mean_iou'][best_idx]
      
      print("Best val_loss: %.4f, val_mean_iou: %.4f"%(val_loss, val_mean_iou))
      loss.append(val_loss)
      metric.append(val_mean_iou)
      
      model = keras.models.load_model('model/keras_%d.model'%fold_index, custom_objects={'mean_iou': mean_iou, 'iou_bce_loss': iou_bce_loss, 'dice_loss': dice_loss})
      pred_test += model.predict(X_test) / fold
      fold_index = fold_index + 1
      
      del model
      gc.collect()
      
      if fast:
        break
      
      
    print('%d fold val_loss: %.4f, val_mean_iou: %.4f'%(fold, np.mean(loss), np.mean(metric)))
    return pred_test, np.mean(loss), np.mean(metric)

  
pred_test, val_loss, val_mean_iou = cv(X_train, y_train, X_test, 5, fast=False)

INFO:tensorflow:Querying Tensorflow master (grpc://10.240.1.2:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 8986511690744612254)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 17578464049263961541)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 1822323957473087955)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 11699108356318831405)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 15578942636750116165)
INFO:tensorflow:*** Available Device: _DeviceAttr

KeyboardInterrupt: ignored

In [2]:
import tensorflow as tf
import numpy as np

def test_model():
  x = tf.keras.layers.Input(shape=(101,101,1), dtype=np.float)
  y = tf.keras.layers.Conv2D(1,1, padding='same', activation='sigmoid', dtype=np.float)(x)
  m = tf.keras.Model(x, y)
  return m

model = test_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 101, 101, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 101, 101, 1)       2         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


In [3]:
resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu='kaggle-tpu')
strategy = tf.contrib.tpu.TPUDistributionStrategy(resolver)
model = tf.contrib.tpu.keras_to_tpu_model(model, strategy=strategy)
session_master = resolver.master()

INFO:tensorflow:Querying Tensorflow master (grpc://10.240.1.2:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 6931466752683336584)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 2319523528817134981)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 2621229120846880910)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 13017604127097533172)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 13684817192547276644)
INFO:tensorflow:*** Available Device: _DeviceAttri

In [5]:
import numpy as np


def _set_shape(image, mask):
  image.set_shape([101, 101, 1])
  mask.set_shape([101, 101, 1])
  return image, mask

def input_fn():
  images = tf.random_uniform([1024, 101, 101, 1])
  masks = tf.random_uniform([1024, 101, 101, 1])
  
  dataset = tf.data.Dataset.from_tensor_slices((images, masks))
#   print(dataset)
#   dataset = dataset.map(lambda image, mask: tuple(tf.py_func(random_h_flip, [image, mask], [tf.float32, tf.float32])))
#   print(dataset)
  dataset = dataset.map(_random_flip_left_right)
#   print(dataset)
  dataset = dataset.shuffle(1000).repeat(100000).batch(32, drop_remainder=True)
#   print(dataset)

  return dataset

model.compile(
      optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01),
      loss='binary_crossentropy',
      metrics=['binary_crossentropy'])



model.fit(
    input_fn,
    validation_data=input_fn,
    validation_steps=100,
    steps_per_epoch=100,
    epochs=20)

<TensorSliceDataset shapes: ((101, 101, 1), (101, 101, 1)), types: (tf.float32, tf.float32)>
<TensorSliceDataset shapes: ((101, 101, 1), (101, 101, 1)), types: (tf.float32, tf.float32)>
<MapDataset shapes: ((101, 101, 1), (101, 101, 1)), types: (tf.float32, tf.float32)>
<BatchDataset shapes: ((32, 101, 101, 1), (32, 101, 101, 1)), types: (tf.float32, tf.float32)>
<TensorSliceDataset shapes: ((101, 101, 1), (101, 101, 1)), types: (tf.float32, tf.float32)>
<TensorSliceDataset shapes: ((101, 101, 1), (101, 101, 1)), types: (tf.float32, tf.float32)>
<MapDataset shapes: ((101, 101, 1), (101, 101, 1)), types: (tf.float32, tf.float32)>
<BatchDataset shapes: ((32, 101, 101, 1), (32, 101, 101, 1)), types: (tf.float32, tf.float32)>
Epoch 1/20
INFO:tensorflow:New input shapes; (re-)compiling: mode=train, [TensorSpec(shape=(32, 101, 101, 1), dtype=tf.float32, name=None), TensorSpec(shape=(32, 101, 101, 1), dtype=tf.float32, name=None)]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflo

KeyboardInterrupt: ignored

Epoch 1/10


InvalidArgumentError: ignored

In [1]:
# Copyright 2018 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

r"""ResNet-50 implemented with Keras running on Cloud TPUs.
This file shows how you can run ResNet-50 on a Cloud TPU using the TensorFlow
Keras support. This is configured for ImageNet (e.g. 1000 classes), but you can
easily adapt to your own datasets by changing the code appropriately.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl import flags
from absl import logging
import tensorflow as tf
import numpy as np

import imagenet_input

try:
  import h5py as _  # pylint: disable=g-import-not-at-top
  HAS_H5PY = True
except ImportError:
  logging.warning('`h5py` is not installed. Please consider installing it '
                  'to save weights for long-running training.')
  HAS_H5PY = False


flags.DEFINE_bool('use_tpu', True, 'Use TPU model instead of CPU.')
flags.DEFINE_string('tpu', None, 'Name of the TPU to use.')
flags.DEFINE_string('data', None, 'Path to training and testing data.')

FLAGS = flags.FLAGS

PER_CORE_BATCH_SIZE = 128
NUM_CLASSES = 1000
IMAGE_SIZE = 224
EPOCHS = 90  # Standard imagenet training regime.
APPROX_IMAGENET_TRAINING_IMAGES = 1280000  # Approximate number of images.
APPROX_IMAGENET_TEST_IMAGES = 48000  # Approximate number of images.

WEIGHTS_TXT = '/tmp/resnet50_weights.h5'


def main(argv):
  logging.info('Building Keras ResNet-50 model.')
  model = tf.keras.applications.resnet50.ResNet50(
      include_top=True,
      weights=None,
      input_tensor=None,
      input_shape=None,
      pooling=None,
      classes=NUM_CLASSES)

  num_cores = 8
  batch_size = PER_CORE_BATCH_SIZE * num_cores

  if FLAGS.use_tpu:
    logging.info('Converting from CPU to TPU model.')
    resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu=FLAGS.tpu)
    strategy = tf.contrib.tpu.TPUDistributionStrategy(resolver)
    model = tf.contrib.tpu.keras_to_tpu_model(model, strategy=strategy)
    session_master = resolver.master()
  else:
    session_master = ''

  logging.info('Compiling model.')
  model.compile(
      optimizer=tf.train.GradientDescentOptimizer(learning_rate=1.0),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'])

  if FLAGS.data is None:
    training_images = np.random.randn(
        batch_size, IMAGE_SIZE, IMAGE_SIZE, 3).astype(np.float32)
    training_labels = np.random.randint(NUM_CLASSES, size=batch_size,
                                        dtype=np.int32)
    logging.info('Training model using synthetica data.')
    model.fit(training_images, training_labels, epochs=EPOCHS,
              batch_size=batch_size)
    logging.info('Evaluating the model on synthetic data.')
    model.evaluate(training_images, training_labels, verbose=0)
  else:
    imagenet_train, imagenet_eval = [imagenet_input.ImageNetInput(
        is_training=is_training,
        data_dir=FLAGS.data,
        per_core_batch_size=PER_CORE_BATCH_SIZE)
                                     for is_training in [True, False]]
    logging.info('Training model using real data in directory "%s".',
                 FLAGS.data)
    model.fit(imagenet_train.input_fn,
              epochs=EPOCHS,
              steps_per_epoch=int(APPROX_IMAGENET_TRAINING_IMAGES / batch_size))

    if HAS_H5PY:
      logging.info('Save weights into %s', WEIGHTS_TXT)
      model.save_weights(WEIGHTS_TXT, overwrite=True)

    logging.info('Evaluating the model on the validation dataset.')
    score = model.evaluate(
        imagenet_eval.input_fn,
        steps=int(APPROX_IMAGENET_TEST_IMAGES // batch_size),
        verbose=1)
    print('Evaluation score', score)


if __name__ == '__main__':
  tf.logging.set_verbosity(tf.logging.INFO)
  tf.app.run()

ImportError: ignored

In [0]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import dotenv
# import keras
import os
dotenv.load_dotenv('.env')

!mkdir model
# !gsutil rsync gs://{os.environ['GCP_BUCKET']}/model model

In [7]:
train = pd.read_csv('raw/train.csv').set_index('id')
test = pd.read_csv('raw/sample_submission.csv').set_index('id')
depths = pd.read_csv('raw/depths.csv').set_index('id')

train = pd.merge(train, depths, left_index=True, right_index=True, how='inner')
train.shape, test.shape, depths.shape

((4000, 2), (18000, 1), (22000, 1))

In [0]:
img_size_target = 101

In [9]:
from keras.preprocessing.image import load_img

train["image"] = [np.array(load_img("./raw/images/{}.png".format(idx), grayscale=True)) / 255 for idx in tqdm(train.index)]
train["mask"] = [np.array(load_img("./raw/masks/{}.png".format(idx), grayscale=True)) / 255 for idx in tqdm(train.index)]

X_train = np.array(train['image'].tolist()).reshape(-1, img_size_target, img_size_target, 1)
y_train = np.array(train['mask'].tolist()).reshape(-1, img_size_target, img_size_target, 1)

X_test = np.array([np.array(load_img("./raw/images/{}.png".format(idx), grayscale=True)) / 255 for idx in tqdm(test.index)]).reshape(-1, img_size_target, img_size_target, 1)

Using TensorFlow backend.
  0%|          | 0/4000 [00:00<?, ?it/s]/home/Hoiy/.local/lib/python3.5/site-packages/keras_preprocessing/image.py:492: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|██████████| 18000/18000 [00:11<00:00, 1533.46it/s]


In [10]:
X_train = X_train.astype('float32', copy=False)
y_train = y_train.astype('float32', copy=False)

X_test = X_test.astype('float32', copy=False)

X_train.shape, y_train.shape, X_test.shape

((4000, 101, 101, 1), (4000, 101, 101, 1), (18000, 101, 101, 1))

In [0]:
import tensorflow as tf
# from tensorflow.contrib.tpu.python.tpu import tpu_config
# from tensorflow.contrib.tpu.python.tpu import tpu_estimator
# from tensorflow.contrib.tpu.python.tpu import tpu_optimizer

In [0]:
# input: (x, 101, 101, 1), (x, 101, 101, 1)
# output: (x,)
def batch_iou(y_true, y_pred):
    pred_ = tf.round(y_pred)
    truth_ = y_true
    
    i = tf.reduce_sum(pred_ * truth_, axis=[1,2,3])
    u = tf.reduce_sum((pred_ + truth_) - (pred_ * truth_), axis=[1,2,3])
    
    epsilon = tf.keras.backend.epsilon()
    return (i + epsilon) / (u + epsilon)

# input: (x,)
# output: ()
def sweep_thresholds_mean_iou(y_true, y_pred):
    ious = batch_iou(y_true, y_pred)
    ious = tf.reshape(ious, (-1,1))
    ious = tf.tile(ious, (1,10))
    thresholds = tf.range(0.5, 1.0, delta=0.05)
    
    return tf.reduce_mean(tf.to_float(ious >= thresholds))

def metric_fn(y_true, y_pred):
  return {
    'sweep_thresholds_mean_iou': sweep_thresholds_mean_iou(y_true, y_pred)
  }

In [0]:
# def host_call_fn(gs, loss, lr, ce):
#   """Training host call. Creates scalar summaries for training metrics.
#   This function is executed on the CPU and should not directly reference
#   any Tensors in the rest of the `model_fn`. To pass Tensors from the
#   model to the `metric_fn`, provide as part of the `host_call`. See
#   https://www.tensorflow.org/api_docs/python/tf/contrib/tpu/TPUEstimatorSpec
#   for more information.
#   Arguments should match the list of `Tensor` objects passed as the second
#   element in the tuple passed to `host_call`.
#   Args:
#     gs: `Tensor with shape `[batch]` for the global_step
#     loss: `Tensor` with shape `[batch]` for the training loss.
#     lr: `Tensor` with shape `[batch]` for the learning_rate.
#     ce: `Tensor` with shape `[batch]` for the current_epoch.
#   Returns:
#     List of summary ops to run on the CPU host.
#   """
#   gs = gs[0]
#   # Host call fns are executed FLAGS.iterations_per_loop times after one
#   # TPU loop is finished, setting max_queue value to the same as number of
#   # iterations will make the summary writer only flush the data to storage
#   # once per loop.
#   with summary.create_file_writer(
#       FLAGS.model_dir, max_queue=FLAGS.iterations_per_loop).as_default():
#     with summary.always_record_summaries():
#       summary.scalar('loss', loss[0], step=gs)
#       summary.scalar('learning_rate', lr[0], step=gs)
#       summary.scalar('current_epoch', ce[0], step=gs)

#       return summary.all_summary_ops()

In [0]:
# def test_model_fn(features, labels, mode, params):
#     ###################################################################
#     # The Keras LSTM layer that causes an error unless it is unrolled #
#     ###################################################################
#     predictions = tf.keras.layers.Conv2D(filters=1, kernel_size=1)(features)

#     # Create ops for the TPUEstimatorSpec
#     loss = tf.losses.mean_squared_error(labels=labels, predictions=predictions)
#     optimizer = tf.train.GradientDescentOptimizer(0.001)
# #     if FLAGS.use_tpu:
#     optimizer = tpu_optimizer.CrossShardOptimizer(optimizer)
  
#     global_step = tf.train.get_global_step()
#     train_op = optimizer.minimize(loss, global_step)

#     # Return the TPUEstimatorSpec
#     return tpu_estimator.TPUEstimatorSpec(
#         mode=mode,
#         loss=loss,
#         train_op=train_op)

In [0]:
# # https://github.com/tensorflow/tpu/blob/master/models/experimental/cifar_keras/cifar_keras.py

# def model_fn(features, labels, mode, config, params):
# #   print(features)
#   v = tf.keras.layers.Input(tensor=features)
#   pred = tf.keras.layers.Conv2D(
#     filters=1,
#     kernel_size=3,
#     padding='same',
#     activation='relu'
#   )(v)
  
# #   print(pred)

# #   pred=features
  
# #   if mode == tf.estimator.ModeKeys.TRAIN:
#     # Compute the current epoch and associated learning rate from global_step.
#   global_step = tf.train.get_global_step()

#   loss = tf.keras.losses.binary_crossentropy(labels, pred)  

#   optimizer = tf.train.AdamOptimizer()
#   optimizer = tf.contrib.tpu.CrossShardOptimizer(optimizer)

#   # Batch normalization requires UPDATE_OPS to be added as a dependency to
#   # the train operation.
#   update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
#   with tf.control_dependencies(update_ops):
#     train_op = optimizer.minimize(loss, global_step)
      
# #     model = tf.keras.Model(inputs=features, outputs=pred)
    
#   return tf.contrib.tpu.TPUEstimatorSpec(
#     mode=mode,
#     loss=loss,
#     train_op=train_op,
#     host_call=None,
#     eval_metrics=(metric_fn, [labels, pred]))

In [0]:
# def main(unused_argv=None):
#     tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver('kaggle-tpu')

    
#     config = tf.contrib.tpu.RunConfig(
#       cluster=tpu_cluster_resolver,
#       model_dir='gs://kaggle-195720-tgs-salt-identification-challenge/temp',
# #       save_checkpoints_secs=3600,
# #       session_config=tf.ConfigProto(
# #           allow_soft_placement=True, log_device_placement=True),
# #       tpu_config=tf.contrib.tpu.TPUConfig(
# #           iterations_per_loop=FLAGS.iterations_per_loop,
# #           num_shards=FLAGS.num_shards),
#   )
    
    
# #     config = tpu_config.RunConfig(
# #         master=master_url,
# #         model_dir='gs://kaggle-195720-tgs-salt-identification-challenge/temp')

#     # Create the TPUEstimator
#     estimator = tpu_estimator.TPUEstimator(
#       use_tpu=True,
#       model_fn=model_fn,
#       config=config,
#       train_batch_size=1024)

#     # Train the estimator for 10 steps
#     estimator.train(input_fn, max_steps=1000)

In [0]:
# def input_fn(params):
#     dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
# #     dataset = dataset.apply(tf.contrib.data.batch_and_drop_remainder(params['batch_size']))
    
#     print(dataset)

#     # Make input_fn for the TPUEstimator train step
# #     dataset_fn = dataset.make_one_shot_iterator().get_next()
#     dataset = dataset.shuffle(1000).repeat().batch(params['batch_size'])
    
# #     dataset = dataset.prefetch(1)
#     return dataset
  
  
# # dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
# # dataset
# # dataset = dataset.apply(tf.contrib.data.batch_and_drop_remainder(128))

# # print(dataset)

# # # Make input_fn for the TPUEstimator train step
# # dataset_fn = dataset.make_one_shot_iterator().get_next()

In [0]:
# main()

In [0]:
# import os
# import tensorflow as tf
# from tensorflow.contrib import tpu
# from tensorflow.contrib.cluster_resolver import TPUClusterResolver

# def axy_computation(a, x, y):
#   return a * x + y

# inputs = [
#     3.0,
#     tf.ones([3, 3], tf.float32),
#     tf.ones([3, 3], tf.float32),
# ]

# tpu_computation = tpu.rewrite(axy_computation, inputs)

# tpu_grpc_url = TPUClusterResolver(
#     tpu=['kaggle-tpu']).get_master()

# with tf.Session(tpu_grpc_url) as sess:
#   sess.run(tpu.initialize_system())
#   sess.run(tf.global_variables_initializer())
#   output = sess.run(tpu_computation)
#   print(output)
#   sess.run(tpu.shutdown_system())

# print('Done!')

In [0]:
# import keras
# from keras import Model
# from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
# from keras.models import load_model
# from keras.optimizers import Adam
# from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
# from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, Concatenate, Dropout, BatchNormalization, SpatialDropout2D, UpSampling2D

# from keras import backend as K
import tensorflow as tf


# input: (x, 101, 101, 1), (x, 101, 101, 1)
# output: (x,)
def batch_iou(y_true, y_pred):
    pred_ = tf.round(y_pred)
    truth_ = y_true
    
    i = tf.reduce_sum(pred_ * truth_, axis=[1,2,3])
    u = tf.reduce_sum((pred_ + truth_) - (pred_ * truth_), axis=[1,2,3])
    
    epsilon = tf.keras.backend.epsilon()
    return (i + epsilon) / (u + epsilon)

# input: (x,)
# output: ()
def mean_iou(y_true, y_pred):
    ious = batch_iou(y_true, y_pred)
    ious = tf.reshape(ious, (-1,1))
    ious = tf.tile(ious, (1,10))
    thresholds = tf.range(0.5, 1.0, delta=0.05)
    
    return tf.reduce_mean(tf.to_float(ious >= thresholds))


# def dice_loss(y_true, y_pred):
#     smooth = 1.
#     y_true_f = K.flatten(y_true)
#     y_pred_f = K.flatten(y_pred)
#     intersection = y_true_f * y_pred_f
#     score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
#     return 1. - score
  
  

# # define iou or jaccard loss function
# def jaccard_loss(y_true, y_pred):
#     y_true = tf.reshape(y_true, [-1])
#     y_pred = tf.reshape(y_pred, [-1])
#     intersection = tf.reduce_sum(y_true * y_pred)
    
#     epsilon = tf.keras.backend.epsilon()
    
#     score = (intersection + epsilon) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection + epsilon)
#     return 1 - score


  
# def iou_bce_loss(y_true, y_pred):
#     return 0.5 * keras.losses.binary_crossentropy(y_true, y_pred) + 0.5 * jaccard_loss(y_true, y_pred)
  
  
# def mean_iou(y_true, y_pred):
#     return tf.reduce_mean(tf.to_float(tf.greater(tf.round(y_true), y_pred)))
    

# def mean_iou(y_true, y_pred):
#     i = tf.round(y_pred) * tf.round(y_true)
#     o = tf.round(y_pred) * tf.round(y_true)
#     mean_iou = tf.metrics.mean_iou(y_true, tf.round(y_pred), 1)
#     print(mean_iou)
#     print(tf.tile(mean_iou, 10))
#     print(mean_iou[0])
#     return mean_iou[0]
#     return tf.reduce_mean(tf.tile(mean_iou, 10))
  
# def mean_iou(y_true, y_pred):
#     prec = []
#     for t in np.arange(0.5, 1.0, 0.05):
#         y_pred_ = tf.to_int32(y_pred > t)
#         score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
#         K.get_session().run(tf.local_variables_initializer())
#         with tf.control_dependencies([up_opt]):
#             score = tf.identity(score)
#         prec.append(score)
#     return K.mean(K.stack(prec), axis=0)


def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True):
    x = Conv2D(filters, size, strides=strides, padding=padding)(x)
    x = BatchNormalization()(x)
    if activation == True:
        x = Activation('relu')(x)
    return x

  
def residual_block(blockInput, num_filters=16):
    x = BatchNormalization()(x)
    x = convolution_block(x, num_filters, (3,3) )
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = Add()([x, blockInput])
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x
  
  
def conv_block(m, dim, acti, bn, res, do=0.5):
    n = tf.keras.layers.Conv2D(dim, 3, activation=acti, padding='same')(m)
    n = tf.keras.layers.BatchNormalization()(n) if bn else n
    n = tf.keras.layers.SpatialDropout2D(do)(n) if do else n
    n = tf.keras.layers.Conv2D(dim, 3, activation=acti, padding='same')(n)
    n = tf.keras.layers.BatchNormalization()(n) if bn else n
    n = tf.keras.layers.Concatenate()([m, n]) if res else n
    return n

  
def level_block(m, dim, depth, inc, acti, do, bn, mp, up, res):
    if depth > 0:
        n = conv_block(m, dim, acti, bn, res)
        m = tf.keras.layers.MaxPooling2D()(n) if mp else Conv2D(dim, 3, strides=2, padding='same')(n)
        m = level_block(m, int(inc*dim), depth-1, inc, acti, do, bn, mp, up, res)
        if up:
            m = tf.keras.layers.UpSampling2D()(m)
            m = tf.keras.layers.Conv2D(dim, 2, activation=acti, padding='same')(m)
        else:
#             padding = ['valid', 'same','same','valid','same','valid']
            padding = ['same','same','valid','same','valid']
            m = tf.keras.layers.Conv2DTranspose(dim, 3, strides=2, activation=acti, padding=padding[depth-1])(m)
        n = tf.keras.layers.Concatenate()([n, m])
        m = conv_block(n, dim, acti, bn, res)
    else:
        m = conv_block(m, dim, acti, bn, res, do=do)
    return m

  
# def UNet(img_shape, out_ch=1, start_ch=32, depth=5, inc_rate=2., activation='relu', 
#          dropout=False, batchnorm=True, maxpool=True, upconv=False, residual=True):
#     i = tf.keras.layers.Input(shape=img_shape)
#     o = level_block(i, start_ch, depth, inc_rate, activation, dropout, batchnorm, maxpool, upconv, residual)
#     o = tf.keras.layers.Conv2D(out_ch, 1, activation='sigmoid')(o)
#     return tf.keras.Model(inputs=i, outputs=o)

def UNet(img_shape, out_ch=1, start_ch=32, depth=5, inc_rate=2., activation='relu', 
         dropout=False, batchnorm=True, maxpool=True, upconv=False, residual=True):
    i = tf.keras.layers.Input(shape=img_shape)
    o = level_block(i, start_ch, depth, inc_rate, activation, dropout, batchnorm, maxpool, upconv, residual)
    o = tf.keras.layers.Conv2D(out_ch, 1, activation='sigmoid')(o)
    return tf.keras.Model(inputs=i, outputs=o)
  
  

# def ResNet():
#   i = Input(shape=(img_size_target, img_size_target))
  
#   o = Conv2D(filters=1, kernel_size=1, activation='sigmoid')

In [27]:
def build_train_generator(X_train, y_train, batch_size=32):
    # Ref: https://keras.io/preprocessing/image/
    # we create two instances with the same arguments
    data_gen_args = dict(
      rotation_range=360.,
#       width_shift_range=0.1,
#       height_shift_range=0.1,
      fill_mode='reflect',
      horizontal_flip=True,
      vertical_flip=True
    )
    image_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)

    # Provide the same seed and keyword arguments to the fit and flow methods
    seed = 1
    image_datagen.fit(X_train, augment=True, seed=seed)
    mask_datagen.fit(y_train, augment=True, seed=seed)
    
    image_generator = image_datagen.flow(X_train, batch_size=batch_size, seed=seed)
    mask_generator = mask_datagen.flow(y_train, batch_size=batch_size, seed=seed)

    # combine generators into one which yields image and masks
    return zip(image_generator, mask_generator)


def cv(X_train, y_train, X_test, fold, fast=False):
    from sklearn.model_selection import KFold
    kf = KFold(n_splits=fold, shuffle=True, random_state=42)
    
    loss = []
    metric = []
    pred_test = np.zeros(X_test.shape)
    fold_index = 0
    
    start_ch = 32
    depth = 5
         
    
    for train_index, test_index in kf.split(X_train):      
#       model = UNet((img_size_target,img_size_target,1),start_ch=start_ch,depth=depth,batchnorm=True, dropout=False)
      model = test_model()
      resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu='kaggle-tpu')
      strategy = tf.contrib.tpu.TPUDistributionStrategy(resolver)
      model = tf.contrib.tpu.keras_to_tpu_model(model, strategy=strategy)
      session_master = resolver.master()
      
      model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["binary_crossentropy"])

#       early_stopping = EarlyStopping(patience=20, verbose=1)
#       model_checkpoint = ModelCheckpoint("./model/keras_%d.model"%fold_index, save_best_only=True, verbose=1)
#       reduce_lr = ReduceLROnPlateau(factor=0.5, patience=10, min_lr=0.00001, verbose=1)

      epochs = 20000
      batch_size = 32

      history = model.fit_generator(
          build_train_generator(X_train[train_index], y_train[train_index], batch_size),
          validation_data=(X_train[test_index], y_train[test_index]),
          steps_per_epoch=np.ceil(len(X_train[train_index]) / batch_size), 
          epochs=epochs,
#           callbacks=[early_stopping, model_checkpoint, reduce_lr]
      )
      
      best_idx = np.argmin(history.history['val_loss'])
      val_loss = history.history['val_loss'][best_idx]
      val_mean_iou = history.history['val_mean_iou'][best_idx]
      
      print("Best val_loss: %.4f, val_mean_iou: %.4f"%(val_loss, val_mean_iou))
      loss.append(val_loss)
      metric.append(val_mean_iou)
      
      model = keras.models.load_model('model/keras_%d.model'%fold_index, custom_objects={'mean_iou': mean_iou, 'iou_bce_loss': iou_bce_loss, 'dice_loss': dice_loss})
      pred_test += model.predict(X_test) / fold
      fold_index = fold_index + 1
      
      if fast:
        break
      
      
    print('%d fold val_loss: %.4f, val_mean_iou: %.4f'%(fold, np.mean(loss), np.mean(metric)))
    return pred_test, np.mean(loss), np.mean(metric)

  
pred_test, val_loss, val_mean_iou = cv(X_train, y_train, X_test, 5, fast=False)

INFO:tensorflow:Querying Tensorflow master (grpc://10.240.1.2:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 13486222805198966859)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 9984220072123666814)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 18162437541401842557)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 9211313139234737039)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 4136755912537517626)
INFO:tensorflow:*** Available Device: _DeviceAttri

InvalidArgumentError: ignored

In [0]:
# import matplotlib.pyplot as plt

# # summarize history for loss
# plt.plot(history.history['loss'][1:])
# plt.plot(history.history['val_loss'][1:])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train','Validation'], loc='upper left')
# plt.show()

In [0]:
# best_idx = np.argmin(history.history['val_loss'])
# val_loss = history.history['val_loss'][best_idx]
# val_mean_iou = history.history['val_mean_iou'][best_idx]

In [0]:
# model = keras.models.load_model('model/keras.model', custom_objects={'mean_iou': mean_iou, 'iou_bce_loss': iou_bce_loss})

In [0]:
def RLenc(img, order='F', format=True):
    """
    img is binary mask image, shape (r,c)
    order is down-then-right, i.e. Fortran
    format determines if the order needs to be preformatted (according to submission rules) or not

    returns run length as an array or string (if format is True)
    """
    bytes = img.reshape(img.shape[0] * img.shape[1], order=order)
    runs = []  ## list of run lengths
    r = 0  ## the current run length
    pos = 1  ## count starts from 1 per WK
    for c in bytes:
        if (c == 0):
            if r != 0:
                runs.append((pos, r))
                pos += r
                r = 0
            pos += 1
        else:
            r += 1

    # if last run is unsaved (i.e. data ends with 1)
    if r != 0:
        runs.append((pos, r))
        pos += r
        r = 0

    if format:
        z = ''

        for rr in runs:
            z += '{} {} '.format(rr[0], rr[1])    
        return z[:-1]
    else:
        return runs

In [0]:
# class FasterRle(object):
#     """Perform RLE in paralell."""

#     def __init__(self, num_consumers=2):
#         """Initialize class."""
#         self._tasks = multiprocessing.JoinableQueue()
#         self._results = multiprocessing.Queue()
#         self._n_consumers = num_consumers

#         # Initialize consumers
#         self._consumers = [Consumer(self._tasks, self._results) for i in range(self._n_consumers)]
#         for w in self._consumers:
#             w.start()

#     def add(self, array, startIndex):
#         """Add a task to perform."""
#         self._tasks.put(FasterTask(array, startIndex))

#     def get_results(self):
#         """Close all tasks."""
#         # Provide poison pill
#         [self._tasks.put(None) for _ in range(self._n_consumers)]
#         # Wait for finish
#         self._tasks.join()
#         # Return results
#         singles = []
#         while not self._results.empty():
#             singles.append(self._results.get())
            
#         resultDic = dict()
#         for rles, start in singles:
#             #print('start:', start)
#             for i,rle in enumerate(rles):
#                 #print('i:', i)
#                 resultDic[str(start+i)] = rle
#         return resultDic

In [0]:
def batch(func):
  def batch_func(b):
    return np.array([func(e) for e in tqdm(b)])
  return batch_func

In [0]:
# preds_test = model.predict(X_test)

In [0]:
test['rle_mask'] = batch(RLenc)(np.round(pred_test))

100%|██████████| 18000/18000 [00:58<00:00, 309.34it/s]


In [0]:
SUBMISSION_FILE = 'keras_unet_sc32de5wbecjaccloss_%.4f_%.4f.csv'%(val_loss, val_mean_iou)
test.to_csv(SUBMISSION_FILE)
SUBMISSION_FILE

'keras_unet_sc32de5wbecjaccloss_0.1320_0.7477.csv'

In [0]:
SUBMISSION_MESSAGE='"Keras U-net iou_bce_loss: val_loss: %.4f, val_mean_iou: %.4f"'%(val_loss, val_mean_iou)
!kaggle competitions submit -f '{SUBMISSION_FILE}' -m '{SUBMISSION_MESSAGE}'

Using competition: tgs-salt-identification-challenge
Successfully submitted to TGS Salt Identification Challenge

In [0]:
import dotenv
dotenv.load_dotenv('.env')
!gsutil rsync model gs://{os.environ['GCP_BUCKET']}/model

Building synchronization state...
Starting synchronization...
Copying file://model/keras_0.model [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file://model/keras_1.model [Content-Type=application/octet-stream]...
Copying file://model/keras_2.model [Content-Type=application/octet-stream]...
Copying file://model/keras_3

In [0]:
# import tensorflow as tf
# import numpy as np

# def batch_iou(pred, truth):
#     pred_ = tf.round(pred)
#     truth_ = truth
    
#     i = tf.reduce_sum(pred_ * truth_, axis=[1,2,3])
#     u = tf.reduce_sum((pred_ + truth_) - (pred_ * truth_), axis=[1,2,3])
    
#     epsilon = tf.keras.backend.epsilon()
#     return (i + epsilon) / (u + epsilon)

  
# def mean_iou(pred, truth):
#     ious = batch_iou(pred, truth)
#     ious = tf.reshape(ious, (-1,1))
#     ious = tf.tile(ious, (1,10))
#     thresholds = tf.range(0.5, 1.0, delta=0.05)
    
#     return tf.reduce_mean(tf.to_float(ious >= thresholds))


# BATCH_SIZE = 100
# ones = tf.constant(np.ones((BATCH_SIZE, 101, 101, 1), dtype='float32'))
# zeros = tf.constant(np.zeros((BATCH_SIZE, 101, 101, 1), dtype='float32'))
# half = tf.coonstant(np.zeros(BATCH_SIZE, ))

# with tf.Session() as sess:
  
#   print(mean_iou(ones, zeros).eval())
#   print(mean_iou(ones, ones).eval())
#   print(mean_iou(zeros, ones).eval())
#   print(mean_iou(zeros, zeros).eval())
  

0.0
1.0
0.0
1.0


In [24]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Standard Imports
import numpy as np
import tensorflow as tf

BATCH_SIZE = 128
NUM_CLASSES = 10
EPOCHS = 12

# input image dimensions
IMG_ROWS, IMG_COLS = 28, 28

use_tpu=True


def mnist_model(input_shape):
  """Creates a MNIST model."""
  model = tf.keras.models.Sequential()
  model.add(
      tf.keras.layers.Conv2D(
          32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
  model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(tf.keras.layers.Dropout(0.25))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(128, activation='relu'))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(NUM_CLASSES, activation='softmax'))
  return model


def main(unused_dev):
  use_tpu = True

  print('Mode:', 'TPU' if use_tpu else 'CPU')

  if True:
    print('Using fake data')
    x_train = np.random.random((128, IMG_ROWS, IMG_COLS))
    y_train = np.zeros([128, 1], dtype=np.int32)
    x_test, y_test = x_train, y_train
  else:
    # the data, split between train and test sets
    print('Using real data')
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

  x_train = x_train.reshape(x_train.shape[0], IMG_ROWS, IMG_COLS, 1)
  x_test = x_test.reshape(x_test.shape[0], IMG_ROWS, IMG_COLS, 1)
  input_shape = (IMG_ROWS, IMG_COLS, 1)

  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  x_train /= 255
  x_test /= 255
  print('x_train shape:', x_train.shape)
  print(x_train.shape[0], 'train samples')
  print(x_test.shape[0], 'test samples')

  # convert class vectors to binary class matrices
  y_train = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)
  y_test = tf.keras.utils.to_categorical(y_test, NUM_CLASSES)

  model = mnist_model(input_shape)

  if use_tpu:
    strategy = tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(tpu='kaggle-tpu')
    )
    model = tf.contrib.tpu.keras_to_tpu_model(model, strategy=strategy)

  model.compile(
      loss=tf.keras.losses.categorical_crossentropy,
      optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.05),
      metrics=['accuracy'])

  model.fit(
      x_train,
      y_train,
      batch_size=BATCH_SIZE,
      epochs=EPOCHS,
      verbose=1,
      validation_data=(x_test, y_test))
  score = model.evaluate(x_test, y_test, verbose=0)
  print('Loss for final step:', score[0])
  print('Accuracy ', score[1])


if __name__ == '__main__':
  tf.logging.set_verbosity(tf.logging.INFO)
  tf.app.run()

Mode: TPU
Using fake data
x_train shape: (128, 28, 28, 1)
128 train samples
128 test samples
INFO:tensorflow:Detected old TPUStrategy API.
Train on 128 samples, validate on 128 samples
Epoch 1/12
INFO:tensorflow:New input shapes; (re-)compiling: mode=train, [TensorSpec(shape=(16, 28, 28, 1), dtype=tf.float32, name='conv2d_4_input0'), TensorSpec(shape=(16, 10), dtype=tf.float32, name='dense_5_target0')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for conv2d_4_input
INFO:tensorflow:Started compiling


InvalidArgumentError: ignored